# Data cleaning and pre-processing

This notebook shows the complete process followed to transform the original data set in a new one that contains the business variables with to be analyized in the next phase of the project.

* Data was readed from the provided file `datos.csv` a 3.6 gigabytes CSV file with 36 variables about 10.651.775 bank operations.
* All the variables were converted to its correct type: date, character, categorical or numeric.
* New variables were derived from the original ones
* Finally the data was normalized and centered around it's mean, so the K-means algorithm could calculate the distance between each observation correctly.

The resulting data set was stored in the Apache Hadoop Distributed File System running on the cloud ready to be analyzed with Spark.

In [64]:
# data.table R package provides all the needed data wrangling features to deal with big data files
library('data.table')

In [65]:
# Reading the data file
DT <- fread('./sample.csv', encoding='Latin-1', na.strings=c("","NA"), sep=",")

In [66]:
# First two rows of the new data table
head(DT, 2)

,ANO,MES,DIA,OP_ADQUIRENTE,ADQUIERENTE,DES_TIPO_ADQUIRENTE,OP_EMISOR,EMISOR,DES_TIPO_EMISOR,DES_AMBITO,OP_IDENT_TERMINAL,OP_COD_POST_COMERCIO,DES_PROVINCIA,LOCALIDAD,OP_COD_PAIS_COMERCIO,DES_MARCA,DES_GAMA,DES_PRODUCTO,TIPO_TARJETA,DES_CREDEB,DES_CLASE_OPERACION,DES_PAGO,DES_RESULTADO,PER_ID_PERSONA,PER_TIPO_PERS,PER_FECHA_ALTA,OF_COD_POST,PER_COD_PAIS_NAC,OF_COD_PAIS_RES,PER_ID_SEXO,PER_EST_CIVIL,PER_MARCA_EMP,PER_MARCA_FALL,PER_FECHA_NAC,NOPER,IMPOPER
1,2016,03,26,UBN3YFFD8J1TZLDX4IV2,6CVN1DBRKUOUTFT8VBXJ,EURO 6000,CM8GMN7BQOF9JJ1XXCPE,KSPHEXET1G2LNR4OXAGU,EURO 6000,On us,EUNDITVNRWGASN0VCFMK,99999,NO EXISTE LA PROVINCIA,NA,ESP,MasterCard,EstÃ¡ndar,MasterCard,P,DÃ©bito,Reintegros,Debito,OK,HWG2BTMLK1WMRVJT3TMU,F,19840926,29002,ESP,ESP,F,C,0,0,19410304,1,30
2,2016,03,03,BM3MV1QJ1RWI6XB8W36S,Q4SRXYQNPFB8ST2BCSLT,EURO 6000,X4W6L75KAK6TKQFMYXGJ,T08GEJ1FFLLW5WK82Z0M,EURO 6000,On us,HDOHII3H5SMPCZIJI07V,99999,NO EXISTE LA PROVINCIA,NA,ESP,Visa,EstÃ¡ndar,Electron/Plus/V Pay,P,DÃ©bito,Reintegros,Debito,OK,S72O8TEU3YWJFJLXJO7Y,F,19931218,47006,ESP,ESP,F,C,0,0,19451121,1,200


In [67]:
# All the variables were readed as characters by fread data table function so we need to recode them.
as.data.frame(sapply(DT,class))

,"sapply(DT, class)"
ANO,character
MES,character
DIA,character
OP_ADQUIRENTE,character
ADQUIERENTE,character
DES_TIPO_ADQUIRENTE,character
OP_EMISOR,character
EMISOR,character
DES_TIPO_EMISOR,character
DES_AMBITO,character


In [ ]:
# Define a new variable FECHA with the operation date
DT[,FECHA:=as.Date(paste(ANO, MES, DIA, sep="-" ), tz = "Europe/Madrid")]

# Convert to date format both PER_FECHA_NAC and PER_FECHA_ALTA
DT[,PER_FECHA_NAC:=as.Date(PER_FECHA_NAC, format = "%Y%m%d", tz = "Europe/Madrid")]
DT[,PER_FECHA_ALTA:=as.Date(PER_FECHA_ALTA, format = "%Y%m%d", tz = "Europe/Madrid")]
head(DT,2)

In [ ]:
# Convert character variables into categorical (factor) variables in order to speed data processing
variables <- c('ANO','MES','DIA','OP_ADQUIRENTE','DES_TIPO_EMISOR','DES_PROVINCIA', 'DES_TIPO_ADQUIRENTE', 'DES_AMBITO', 'OP_COD_PAIS_COMERCIO','DES_MARCA','DES_GAMA','DES_PRODUCTO', 'TIPO_TARJETA', 'DES_CREDEB','DES_CLASE_OPERACION', 'DES_PAGO','DES_RESULTADO','PER_TIPO_PERS','PER_COD_PAIS_NAC', 'OF_COD_PAIS_RES','PER_ID_SEXO','PER_EST_CIVIL','PER_MARCA_EMP','PER_MARCA_FALL')
DT[,(variables):=lapply(.SD, as.factor),.SDcols=variables]
rm(variables)

In [ ]:
# convert into numeric format
variables <- c('NOPER','IMPOPER')
DT[,(variables):=lapply(.SD, as.numeric),.SDcols=variables]
rm(variables)

In [71]:
# Assign NA to DES_PROVINCIA variable with value 'NO EXISTE LA PROVINCIA'
levels(DT$DES_PROVINCIA)[levels(DT$DES_PROVINCIA)=='NO EXISTE LA PROVINCIA'] <- NA

In [72]:
# We reorder de columns to put the date at the begining of each observation
setcolorder(DT, c(ncol(DT), 1:(ncol(DT)-1)))
str(DT)

Classes ‘data.table’ and 'data.frame':	500 obs. of  37 variables:
 $ FECHA               : Date, format: "2016-03-26" "2016-03-03" ...
 $ ANO                 : Factor w/ 1 level "2016": 1 1 1 1 1 1 1 1 1 1 ...
 $ MES                 : Factor w/ 3 levels "01","02","03": 3 3 3 1 3 1 1 1 2 1 ...
 $ DIA                 : Factor w/ 31 levels "01","02","03",..: 26 3 20 5 12 5 20 3 5 29 ...
 $ OP_ADQUIRENTE       : Factor w/ 27 levels "0A86SJKPC5VI5TUKAN01",..: 21 11 25 21 8 21 19 12 2 21 ...
 $ ADQUIERENTE         : chr  "6CVN1DBRKUOUTFT8VBXJ" "Q4SRXYQNPFB8ST2BCSLT" "KTXGB1YGLNA5A3MBQWPC" "6CVN1DBRKUOUTFT8VBXJ" ...
 $ DES_TIPO_ADQUIRENTE : Factor w/ 4 levels "Eufiserv","EURO 6000",..: 2 2 3 2 2 2 2 3 3 2 ...
 $ OP_EMISOR           : chr  "CM8GMN7BQOF9JJ1XXCPE" "X4W6L75KAK6TKQFMYXGJ" "S33T9PXGBMI7GUX051OC" "CM8GMN7BQOF9JJ1XXCPE" ...
 $ EMISOR              : chr  "KSPHEXET1G2LNR4OXAGU" "T08GEJ1FFLLW5WK82Z0M" "JFD7RA18JJ6YI9L720RE" "KSPHEXET1G2LNR4OXAGU" ...
 $ DES_TIPO_EMISOR     : Factor w/ 6

In [ ]:
# Transform coded OP_ADQUIRENTE and ADQUIERENTE into a more human friendly string
setkey(DT,OP_ADQUIRENTE)

op_adquiriente <- seq(from = 1000, to=length(unique(DT$OP_ADQUIRENTE))+999, by =1)
DT[,OP_ADQUIRENTE:=factor(OP_ADQUIRENTE,labels=op_adquiriente)]

adquiriente <- paste("Entidad", op_adquiriente)
DT[,ADQUIERENTE:=factor(ADQUIERENTE,labels=adquiriente)]
head(DT, 2)

In [74]:
# Filter out all the operations where PER_ID_PERSONA is NA and DES_RESULTADO is not OK
DT <- DT[!is.na(PER_ID_PERSONA) & DES_RESULTADO == "OK"]

In [75]:
# Save the data table into a serialized object for later usage. This will speed up later porcesses of merging the resulting data set with the cluster variable with the original one.
saveRDS(DT, file = "./data/DT.rds", compress = T)

## Feature creation
New variables are derived from the previous ones. These variables are the final ones after an iterative process of pruning:
* High correlated variables
* Zero variance predictors or with linear dependencies: that is, variables with few unique values relative to other values within feature, or near zero variance or high frequency of most common value or mode value.

In [76]:
setkeyv(DT,c("PER_ID_PERSONA","MES"))
DT <- DT[, list(F1=median(na.omit(IMPOPER[which(DES_AMBITO == "On us")])),
                F2=median(na.omit(IMPOPER[which(DES_AMBITO == "Inter-Sistemas")])),
                F3=median(na.omit(IMPOPER[which(DES_AMBITO == "Intra-Sistema")]), na.rm = T),
                F4=length(unique(.N[which(DES_AMBITO == "On us")])),
                F5=length(unique(.N[which(DES_AMBITO == "Inter-Sistemas")])),
                F6=length(unique(.N[which(DES_AMBITO == "Intra-Sistema")]))               
),
by=.(PER_ID_PERSONA)]

In [77]:
# Remove any NA observation. Altough none of the previous derived features have NA values. Just for be sure.
DT[is.na(DT)] <- 0

In [78]:
# The data set obtained frim previous operations
head(DT, 2)

,PER_ID_PERSONA,F1,F2,F3,F4,F5,F6
1,010BPUWEHI43K9AB179N,400,0,0,1,0,0
2,0GQ3NQMI1N24JE0499AY,10,0,0,1,0,0


## Data normalization


In [79]:
# data.table data frames doesn't suppor assigning row names. This is a cool and interesting feature in R data frames. We can assign each row an unique labes with the PER_ID_PERSONA and join later the assigned cluster to each PER_ID_PERSONA
# Convert the data.table to a data.frame
DT <- as.data.frame(DT)
# Set each row names as the PER_ID_PERSONA 
row.names(DT) <- DT$PER_ID_PERSONA
# Delete de PER_ID_PERSONA variable, no needed for the clustering algorithm
DT$PER_ID_PERSONA <- NULL

In [80]:
# Data normalization
DT <- scale(DT)
head(DT, 2)

,F1,F2,F3,F4,F5,F6
010BPUWEHI43K9AB179N,2.1703465,-0.1020868,-0.2631964,0.5349538,-0.2944623,-0.4218309
0GQ3NQMI1N24JE0499AY,-0.5424624,-0.1020868,-0.2631964,0.5349538,-0.2944623,-0.4218309


## Export

In [81]:
# write de data.frame as a CSV file ready to be uploaded clustered
write.table(DT[complete.cases(DT),], "./data/sample_scaled.csv", row.names = T, col.names = FALSE, sep=",")